#데이터추출 쿼리

##pymysql설치

In [ ]:
pip install pymysql

In [ ]:
import pymysql
#데이터베이스 연동
conn = pymysql.connect(host='tkt-db-ko.cnirlhwrm55r.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='tkt202202', db='topic_keywords_db', charset='utf8')
cursor = conn.cursor()

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

In [ ]:
print(date)

2022-02-27


''' 레이블은 레이블대로, 데이터는 데이터대로 리스트 파일에 담아야 차트에 바로 적용 가능 '''

##daily_top25

In [ ]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
result = dict(cursor.fetchall())
#오늘에 대한 정보 없을 시 어제날짜로 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '러시아', '산불', '방역', '푸틴', '진화', '합천', '중단', '패스', '민간', '정부', '이재명', '확진', '제재', '고령', '전쟁', '통합', '지역', '협상', '포격', '대선', '논란', '집회', '대통령', '교체']
[31, 29, 26, 17, 16, 16, 15, 14, 14, 14, 13, 12, 12, 11, 11, 10, 10, 10, 9, 9, 8, 8, 7, 7, 7]


##weekly_top25

In [ ]:
#DB keyword테이블에서 이번주 기준 keyword별 언급량 합계 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = dict(cursor.fetchall())
#이번주에 대한 정보가 없을시 지난주에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '키예프', '러시아', '강풍', '최다', '푸틴', '산불', '방역', '제재', '항전', '역대', '수도', '사망자', '윤석열', '확진', '별세', '대선', '평화', '지성', '대통령', '시대', '강원', '러시아군', '장관', '직접']
[28, 24, 18, 14, 12, 11, 11, 10, 9, 9, 9, 9, 9, 9, 9, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6]


##monthly_top25

In [ ]:
#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format(curdate(), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = dict(cursor.fetchall())
#이번달에 대한 정보가 없을시 지난달에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '러시아', '산불', '방역', '푸틴', '진화', '합천', '중단', '패스', '민간', '정부', '이재명', '확진', '제재', '고령', '전쟁', '통합', '지역', '협상', '포격', '대선', '논란', '집회', '대통령', '교체']
[31, 29, 26, 17, 16, 16, 15, 14, 14, 14, 13, 12, 12, 11, 11, 10, 10, 10, 9, 9, 8, 8, 7, 7, 7]


##daily_chart

In [ ]:
#DB keyword테이블에서 어제, 오늘 키워드 카운트 순 상위 25개의 keyword와 weight추출
#오늘 top1~25출력
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
today = dict(cursor.fetchall())
if len(today) != 0:
  #전날 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())
else:
  #오늘 정보가 없을 시 어제,그저께 정보로 데이터 출력
  #어제 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  today = dict(cursor.fetchall()) 
  #그저께 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -2 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())

daily_chart = []

#어제 비교, 오늘 키워드의 등락표시
for i in today.keys():
    if i in yesterday.keys():
      if today[i] > yesterday[i]:
          daily_chart.append([i, today[i], "up"])
      elif today[i] == yesterday[i]:
          daily_chart.append([i, today[i], "same"])
      else:
          daily_chart.append([i, today[i], "down"])
    else:
      daily_chart.append([i, today[i], "new"])

dic_data = {
    'data': daily_chart
}
print(dic_data)

{'data': [['우크라', 31, 'down'], ['러시아', 29, 'up'], ['산불', 26, 'up'], ['방역', 17, 'new'], ['푸틴', 16, 'new'], ['진화', 16, 'new'], ['합천', 15, 'new'], ['중단', 14, 'new'], ['패스', 14, 'new'], ['민간', 14, 'new'], ['정부', 13, 'new'], ['이재명', 12, 'new'], ['확진', 12, 'up'], ['제재', 11, 'up'], ['고령', 11, 'new'], ['전쟁', 10, 'up'], ['통합', 10, 'new'], ['지역', 10, 'new'], ['협상', 9, 'down'], ['포격', 9, 'new'], ['대선', 8, 'same'], ['논란', 8, 'new'], ['집회', 7, 'new'], ['대통령', 7, 'new'], ['교체', 7, 'new']]}


##weekly_chart

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번주(월~일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT ADDDATE(curdate(), - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < ADDDATE(curdate(), - WEEKDAY(curdate())+ 6)) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
day = cursor.fetchall()

if len(keywords) == 0:
  #이번주 데이터가 없을 시 지난주 데이터 추출
  #DB keyword테이블에서 지난주 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난주(월~일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < DATE(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())))) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

weekly_chart = {}
#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * 7
  no = 0
  for d in day:
    if d[0] == date:
      break
    else:
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[no] = daily_weight[0][0]
      no += 1
  weekly_chart[keywords[i][0]] = weight
print(weekly_chart)

{'우크라': [35, 31, 0, 0, 0, 0, 0], '러시아': [28, 29, 0, 0, 0, 0, 0], '산불': [8, 26, 0, 0, 0, 0, 0], '협상': [14, 9, 0, 0, 0, 0, 0], '윤석열': [14, 6, 0, 0, 0, 0, 0]}


##monthly_chart

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번달(1일~말일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY(NOW()))SELECT date_format(DAY, '%Y-%m-%d') FROM day")
day = cursor.fetchall()

if len(keywords) == 0:
  #이번주 데이터가 없을 시 지난달 데이터 추출
  #DB keyword테이블에서 지난달 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난달(1일~말일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB((curdate() - INTERVAL 1 MONTH), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY((curdate() - INTERVAL 1 MONTH)))SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

monthly_chart = {}

#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * int(day[-1][0][-2:])
  for d in day:
    if d[0] == date:
      break
    else:
      #일별 추출한 keywords의 weight 추출
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[int(d[0][-2:])-1] = daily_weight[0][0]
  monthly_chart[keywords[i][0]] = weight
print(weekly_chart)

{'우크라': [35, 31, 0, 0, 0, 0, 0], '러시아': [28, 29, 0, 0, 0, 0, 0], '산불': [8, 26, 0, 0, 0, 0, 0], '협상': [14, 9, 0, 0, 0, 0, 0], '윤석열': [14, 6, 0, 0, 0, 0, 0]}


''' 레이블은 레이블대로, 데이터는 데이터대로 리스트 파일에 담아야 차트에 바로 적용 가능 '''

##daily_twitter

In [ ]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date(now()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

daily_twitter = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            daily_twitter[word[0]] = tweet[1:]
    no += 1

labels = []
data = []

for i in range(len(daily_twitter.keys())):
    key = list(daily_twitter.keys())[i]
    value = list(daily_twitter.values())[i]
    value = list(value)
    value.insert(0, key)
    labels.append(i + 1)
    data.append(value)

print(labels)
print(data)

[1, 2, 3, 4, 5]
[['우크라', 43530, 43, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'], ['러시아', 257924, 155, 'PrayForUkraine,SBS뉴스'], ['단일', 8105, 742, '안철수소신투표, 기호4번'], ['윤석열', 59412, 4, '양아치,윤석열,팩트체크'], ['결렬', 3473, 71, '대선, 단일화, 윤석열, TV토론, 안철수,안철수소신투표, 기호4번']]


##weekly_twitter

In [ ]:
#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #이번주 정보 없을 시 지난주 정보출력
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%y-%m-%d') AND SUBSTR(keyword_id, 1, 8) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%y-%m-%d') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

#twitter테이블에서 추출한 정보 keyword별로 합치기
weekly_twitter = {}
for i in keywords:
  weekly_twitter[i[0]] = [0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            weekly_twitter[word[0]][0] += tweet[1]
            weekly_twitter[word[0]][1] += tweet[2]
            weekly_twitter[word[0]][2] += tweet[3]
    no += 1
print(weekly_twitter)

{'우크라': [43346, 43, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'], '키예프': [139563, 6, 'SBS뉴스'], '러시아': [233142, 150, 'PrayForUkraine,SBS뉴스'], '강풍': [108341, 22, 'SBS뉴스,날씨, 오늘날씨,뉴스, 최신뉴스,연예,철도기상'], '최다': [124626, 10, 'rt,남자명품추천, 남자벨트추천, 여자가방추천,남자벨트추천, 샤넬가방, 셀린느가방,명품백, 여성가방, 미니백, 샤넬백,문재인케어, 문재인케어,에스펄몰, 샤넬,임영웅, LimYoungWoong, 임영웅사랑은늘도망가, 임영웅유튜브, 임영웅영웅시대,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅_브평UP,임영웅, limyoungwoong, 임영웅유튜브, 임영웅_음원강자, 임영웅_독보적, 임영웅_뮤빗,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅뮤빗어워즈,트와이스, TWICE,펜디미니바게트, 디올몽테인백']}


##monthly_twitter

In [ ]:
# #DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난달 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 5) = date_format((curdate() - INTERVAL 1 MONTH), '%y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

#twitter테이블에서 추출한 정보 keyword별로 합치기
monthly_twitter = {}
for i in keywords:
  monthly_twitter[i[0]] = [0, 0, '']

no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            monthly_twitter[word[0]][0] += tweet[1]
            monthly_twitter[word[0]][1] += tweet[2]
            monthly_twitter[word[0]][2] += tweet[3]
    no += 1
print(monthly_twitter)

{'우크라': [7651, 57, 'SBS뉴스,오늘의신문_스크랩'], '러시아': [204707, 33, '레이시즘_퍼레이드,삼일절, 독립운동, 대한민국만세'], '산불': [95147, 37, '2022산불상황,mbc,SBS뉴스,조건만남'], '방역': [214313, 57, '빅톤, VICTON, 수빈, 정수빈, Subin,상주시청, 재난문자'], '진화': [173150, 3, '2022산불상황,JINHWA_RAID, BLITZERS, 블리처스, JINHWA, 진화,SBS뉴스,ポケモンSV, 第9世代, PokemonDay, PokemonDay2022, ニャオハ, animation,청색경보']}


##daily_youtube

In [ ]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = curdate() GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

daily_youtube = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            daily_youtube[word[0]] = data[1:]
    no += 1
print(daily_youtube)

{'우크라': (8249461, 72202, 32258, 'source:영상, type:방송, genre:사회, format:기타, person:박기완,source:영상, type:방송, genre:국제, format:인터뷰, person:호준석,source:영상, type:방송, genre:국제, format:기타,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,YTN실시간, YTN, 우크라이나, 미국, 러시아, 영국, 사이버공격, 비상사태,SBSBiz, 경제, 뉴스,source:영상, type:방송, genre:국제, format:기타, person:구수본,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 러시아, 우크라이나, 키예프, 전쟁, 블라디미르 푸틴, 볼로디미르 젤렌스키,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 볼로디미르 젤렌스키, 우크라이나, 러시아, 블라디미르 푸틴,sourc'), '러시아': (28240321, 152542, 30699, '금융, 제재, 핵폭탄,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,슈카, 슈카월드, 경제, 시사, 이슈, 뉴스, 상식, 면접, 논술, 자소서, 취업, 주식, 증시, 주가, 시장, 자산, 투자, 금리,source:영상, type:방송, genre:국제, format:기타, person:김원배,source:영상, type:방송, genre:국제, format:리포트, person:기정훈,KOREA NEWS, 

##weekly_youtube

In [ ]:
#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번주 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())  
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%y-%m-%d') AND SUBSTR(keyword_id, 1, 8) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%y-%m-%d') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

#youtube테이블에서 추출한 정보 keyword별로 합치기
weekly_youtube = {}
for i in keywords:
  weekly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            weekly_youtube[word[0]][0] += data[1]
            weekly_youtube[word[0]][1] += data[2]
            weekly_youtube[word[0]][2] += data[3]
            weekly_youtube[word[0]][3] += data[4]
    no += 1
print(weekly_youtube)

{'우크라': [3986689, 33748, 19193, 'JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영, 러시아, 러시아군, 키예프, 우크라이나 침공, 러시아 침공, 푸틴, 바이든, 젤렌스키, 우크라이나 상황, No War Please, 우크라이나 필사 저항, 경비대, 도시 폭격, 러시아 군함 꺼져라,source:영상, type:방송, genre:국제, format:기타, person:김종균,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 전쟁, 러시아,KBS, kbs news, kbs 뉴스, kbs뉴스, 국제, 군사작전, 뉴스, 뉴스_기타, 대통령, 돈바스, 러시아, 사적, 선포, 우크라이나, 움직임, 유지향, 임박, 침공, 푸틴,KBS, 뉴스, KBS뉴스라이'], '키예프': [14004086, 66218, 14307, 'MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,source:영상, type:방송, genre:국제, format:기타, person:호준석,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁,러시아, 우크라, 함락,source:영상, type:방송, genre:국제, format:기타, person:이상순,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스

##monthly_youtube

In [ ]:
#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall()) 
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 5) = date_format((curdate() - INTERVAL 1 MONTH), '%y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall() 

#youtube테이블에서 추출한 정보 keyword별로 합치기
monthly_youtube = {}
for i in keywords:
  monthly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            monthly_youtube[word[0]][0] += data[1]
            monthly_youtube[word[0]][1] += data[2]
            monthly_youtube[word[0]][2] += data[3]
            monthly_youtube[word[0]][3] += data[4]
    no += 1
print(monthly_youtube)

{'우크라': [13203049, 124870, 37345, 'source:영상, type:방송, genre:국제, format:기타, person:한연희,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 대통령, 러시아, 바실렌코, 우크라, 의원,러시아, 우크라이나, 푸틴,MBN, 굿모닝 MBN, MBN뉴스, 우크라이나 침공, 러시아, 우크라이나, 키예프, 우크라이나 사망자, 벨라루스, 하르키우, 유럽연합, EU, 피란민, 러시아 우크라이나 회담, 젤렌스키 대통령, 폴란드, 신변 안전, 우크라이나 민간인 사망,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁, 탱크, 미사일,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, st'], '러시아': [35891026, 268587, 38091, 'MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,러시아, 북한, 정부제재,russia ukraine conflict live, ukraine news live, russia ukraine conflict live updates, russia ukraine war live, ukraine russia co